In [29]:
from app.core.models.car import Car
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from app.core.services.dynaconf import settings
from app.core.models.base import Base

SQLALCHEMY_DATABASE_URL = f"postgresql://{settings.db.user}:{settings.db.password}@{settings.db.host}:{settings.db.port}/{settings.db.db_name}"

engine = create_engine(SQLALCHEMY_DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = SessionLocal()

In [30]:
from app.data import *
import pandas as pd

In [35]:
data = pd.read_csv('car_prices.csv', on_bad_lines='skip')

In [36]:
data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,"kia motors america, inc",20500,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,"kia motors america, inc",20800,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,4.5,1331.0,gray,black,financial services remarketing (lease),31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,4.1,14282.0,white,black,volvo na rep/world omni,27500,27750,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,4.3,2641.0,gray,black,financial services remarketing (lease),66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [38]:
# İlk 100 satırı al
data_subset = data.head(100)

# Yeni bir CSV dosyası olarak kaydet
data_subset.to_csv('first_100_car_prices.csv', index=False)

In [26]:
from app.schemas import CarSchema

selected_columns = ['year', 'make', 'model', 'trim', 'body', 'transmission', 'state',
       'condition', 'odometer', 'color', 'interior', 'price']

data = data.rename(columns={"sellingprice": "price"})
data = data.astype({'condition': 'str'})
data.ffill(inplace=True)

bulk_data = []
for _, row in data.iterrows():
    car = CarSchema(**row)
    bulk_data.append(car)


In [27]:
len(bulk_data)


558811

In [28]:
session.bulk_insert_mappings(Car, bulk_data)
session.commit()